In [76]:
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import string
from nltk import pos_tag
import random
from nltk import NaiveBayesClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd

C:\Users\priyansh\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [38]:
data = pd.read_csv("train.csv")
d2 = pd.read_csv("test.csv")
d2

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,569682010270101504,American,NaN,zsalim03,NaN,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada)
1,569608307184242688,American,NaN,sa_craig,NaN,0,"@AmericanAir after all, the plane didn’t land ...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada)
2,567879304593408001,Southwest,NaN,DanaChristos,NaN,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada)
3,569757651539660801,US Airways,NaN,rossj987,NaN,0,@USAirways I can legitimately say that I would...,NaN,2015-02-22 23:16:24 -0800,"Washington, D.C.",Eastern Time (US & Canada)
4,569900705852608513,American,NaN,tranpham18,NaN,0,@AmericanAir still no response from AA. great ...,NaN,2015-02-23 08:44:51 -0800,New York City,Eastern Time (US & Canada)
5,569235062862036992,United,NaN,setfive,NaN,0,@united we have developers flying down tmrw mo...,NaN,2015-02-21 12:39:49 -0800,"Central Sq. Cambridge, MA",Eastern Time (US & Canada)
6,569824422279950336,US Airways,NaN,earthXplorer,NaN,1,@USAirways hello??? Anyone there?,NaN,2015-02-23 03:41:43 -0800,"Miami, Fl. USA",Eastern Time (US & Canada)
7,567880416201293824,US Airways,NaN,AliNHamdani,NaN,0,@USAirways @husainhaqqani Mr. Husain u shld pr...,NaN,2015-02-17 18:56:56 -0800,Islamabad,Islamabad
8,570021943899877377,US Airways,NaN,BradleyPollock,NaN,0,"@USAirways not likely, flightaware says plane ...",NaN,2015-02-23 16:46:36 -0800,NaN,NaN
9,569690664029462529,American,NaN,kaps12,NaN,0,@AmericanAir they don't even give an option to...,NaN,2015-02-22 18:50:13 -0800,NaN,NaN


In [39]:
x = data["text"].values
y = data["airline_sentiment"].values
x2 = d2["text"].values
x

array(['@SouthwestAir I am scheduled for the morning, 2 days after the fact, yes..not sure why my evening flight was the only one Cancelled Flightled',
       '@SouthwestAir seeing your workers time in and time out going above and beyond is why I love flying with you guys. Thank you!',
       '@united Flew ORD to Miami and back and  had great crew, service on both legs. THANKS',
       ..., '@usairways the. Worst. Ever. #dca #customerservice',
       '@nrhodes85: look! Another apology. DO NOT FLY @USAirways',
       '@united you are by far the worst airline. 4 plane delays on 1 round trip flight. How is that possible.'],
      dtype=object)

In [40]:
def get_simple_pos(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [41]:
stop = stopwords.words('english')
punc = list(string.punctuation)
stop = stop + punc
stop.append('flight')
stop.append('plane')
stop.append('airline')
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [37]:
import re
DATA = "Hey, you - what are you doing here!?"
re.findall(r"[\w']+", DATA)

['Hey', 'you', 'what', 'are', 'you', 'doing', 'here']

In [67]:
x2

array(["@AmericanAir In car gng to DFW. Pulled over 1hr ago - very icy roads. On-hold with AA since 1hr. Can't reach arpt for AA2450. Wat 2 do?",
       '@AmericanAir after all, the plane didn’t land in identical or worse) conditions at GRK according to METARs.',
       "@SouthwestAir can't believe how many paying customers you left high and dry with no reason for flight Cancelled Flightlations Monday out of BDL! Wow.",
       ...,
       'Nice RT @VirginAmerica: The man of steel might be faster, but we have WiFi – just saying. #ScienceBehindTheExperience http://t.co/FGRbpAZSiX',
       '@AmericanAir Aww Thanks AA..DFW was on GMA up here this AM..so i understand ..Btw A.A is my Airline when im able to trv..Love you guys.:)',
       '@united the lounge tells us they have no pillows for my grandma as one of the ladies opens the closet and I see 2 right there. #unitedlies'],
      dtype=object)

In [43]:
def splitfun(x):
    for i in range(x.shape[0]):
        x[i] = re.findall(r"[\w']+", x[i])
    return x

In [44]:
x = splitfun(x)
x

array([list(['SouthwestAir', 'I', 'am', 'scheduled', 'for', 'the', 'morning', '2', 'days', 'after', 'the', 'fact', 'yes', 'not', 'sure', 'why', 'my', 'evening', 'flight', 'was', 'the', 'only', 'one', 'Cancelled', 'Flightled']),
       list(['SouthwestAir', 'seeing', 'your', 'workers', 'time', 'in', 'and', 'time', 'out', 'going', 'above', 'and', 'beyond', 'is', 'why', 'I', 'love', 'flying', 'with', 'you', 'guys', 'Thank', 'you']),
       list(['united', 'Flew', 'ORD', 'to', 'Miami', 'and', 'back', 'and', 'had', 'great', 'crew', 'service', 'on', 'both', 'legs', 'THANKS']),
       ...,
       list(['usairways', 'the', 'Worst', 'Ever', 'dca', 'customerservice']),
       list(['nrhodes85', 'look', 'Another', 'apology', 'DO', 'NOT', 'FLY', 'USAirways']),
       list(['united', 'you', 'are', 'by', 'far', 'the', 'worst', 'airline', '4', 'plane', 'delays', 'on', '1', 'round', 'trip', 'flight', 'How', 'is', 'that', 'possible'])],
      dtype=object)

In [45]:
lemmatizer = WordNetLemmatizer()
x

array([list(['SouthwestAir', 'I', 'am', 'scheduled', 'for', 'the', 'morning', '2', 'days', 'after', 'the', 'fact', 'yes', 'not', 'sure', 'why', 'my', 'evening', 'flight', 'was', 'the', 'only', 'one', 'Cancelled', 'Flightled']),
       list(['SouthwestAir', 'seeing', 'your', 'workers', 'time', 'in', 'and', 'time', 'out', 'going', 'above', 'and', 'beyond', 'is', 'why', 'I', 'love', 'flying', 'with', 'you', 'guys', 'Thank', 'you']),
       list(['united', 'Flew', 'ORD', 'to', 'Miami', 'and', 'back', 'and', 'had', 'great', 'crew', 'service', 'on', 'both', 'legs', 'THANKS']),
       ...,
       list(['usairways', 'the', 'Worst', 'Ever', 'dca', 'customerservice']),
       list(['nrhodes85', 'look', 'Another', 'apology', 'DO', 'NOT', 'FLY', 'USAirways']),
       list(['united', 'you', 'are', 'by', 'far', 'the', 'worst', 'airline', '4', 'plane', 'delays', 'on', '1', 'round', 'trip', 'flight', 'How', 'is', 'that', 'possible'])],
      dtype=object)

In [46]:
def clean_review(words):
    output_array=[]
    for w in words:
        if w.lower() not in stop:
            pos = pos_tag(w)
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_array.append(clean_word.lower())
    return output_array

In [32]:
pos = pos_tag('flying')
clean_word = lemmatizer.lemmatize('flying', pos = get_simple_pos(pos[0][1]))
clean_word

'flying'

In [47]:
x_final = [clean_review(words) for words in x]
x_final

[['southwestair',
  'scheduled',
  'morning',
  '2',
  'days',
  'fact',
  'yes',
  'sure',
  'evening',
  'one',
  'cancelled',
  'flightled'],
 ['southwestair',
  'seeing',
  'worker',
  'time',
  'time',
  'going',
  'beyond',
  'love',
  'flying',
  'guy',
  'thank'],
 ['united',
  'flew',
  'ord',
  'miami',
  'back',
  'great',
  'crew',
  'service',
  'leg',
  'thanks'],
 ['southwestair', 'dultch97', "that's", 'horse', 'radish'],
 ['united',
  'ord',
  'delayed',
  'air',
  'force',
  'one',
  'last',
  'sbn',
  '8',
  '20',
  '5',
  'min',
  'landed'],
 ['united',
  'load',
  'us',
  'flying',
  'sardine',
  'knew',
  'pilot',
  '2',
  'hour',
  'late',
  'incompetent',
  'beyond',
  'belief'],
 ['jetblue',
  'stock',
  'response',
  'delays',
  'frustrating',
  'poor',
  'cust',
  'serv',
  'amp',
  'told',
  '3',
  'ppl',
  'wait',
  'amp',
  "they'd",
  'come',
  'back'],
 ['jetblue',
  "that'd",
  'nice',
  'hoping',
  'rack',
  'enough',
  'mile',
  'take',
  'trip',
  'se

In [54]:
xffinal = [clean_review(words) for words in x_final]
#xffinal.shape

AttributeError: 'list' object has no attribute 'shape'

In [50]:
text_docs = [" ".join(word) for word in xffinal]
text_docs[0]

'southwestair scheduled morning 2 days fact yes sure evening one cancel flightled'

In [52]:
y = [word for word in y]
y

['negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'neutral',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'positive',
 'neutral',
 'negative',
 'negative',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'neutral',
 'positive',
 'positive',
 'negative',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'neutral',
 'negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'neutral',
 'positive'

In [53]:
x_train, x_test, y_train, y_test = train_test_split(text_docs, y)

In [80]:
count_vec = CountVectorizer(max_features = 2000)
x_train_features = count_vec.fit_transform(text_docs)
x_train_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [56]:
count_vec.get_feature_names()

['00',
 '000',
 '10',
 '100',
 '1000',
 '10pm',
 '11',
 '12',
 '13',
 '130',
 '14',
 '140',
 '15',
 '150',
 '16',
 '17',
 '18',
 '19',
 '1hr',
 '1k',
 '1st',
 '20',
 '200',
 '2014',
 '2015',
 '21',
 '22',
 '23',
 '24',
 '24hrs',
 '25',
 '26',
 '27',
 '28',
 '2day',
 '2days',
 '2hrs',
 '2nd',
 '2x',
 '30',
 '300',
 '30am',
 '30pm',
 '32',
 '35',
 '36',
 '39',
 '3am',
 '3hrs',
 '3rd',
 '40',
 '400',
 '42',
 '45',
 '45min',
 '47',
 '48',
 '4th',
 '50',
 '500',
 '50pm',
 '55',
 '59',
 '5hrs',
 '5th',
 '60',
 '600',
 '70',
 '700',
 '719',
 '728',
 '737',
 '75',
 '777',
 '787',
 '80',
 '800',
 '8pm',
 '90',
 '99',
 '9pm',
 'a320',
 'aa',
 'abc',
 'able',
 'absolute',
 'absolutely',
 'absurd',
 'accept',
 'acceptable',
 'accepted',
 'accepting',
 'access',
 'accident',
 'accommodate',
 'accommodating',
 'accommodation',
 'according',
 'account',
 'acct',
 'across',
 'act',
 'action',
 'actual',
 'actually',
 'add',
 'added',
 'adding',
 'additional',
 'address',
 'addressed',
 'admiral',
 'ad

In [57]:
x_test_features = count_vec.transform(x_test)

In [61]:
svc = SVC(kernel = 'linear', C=6)
svc.fit(x_train_features, y_train)

SVC(C=6, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [82]:
rfc = RandomForestClassifier()
rfc.fit(x_train_features,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [78]:
rfc.score(x_test_features,y_test)

0.736976320582878

In [63]:
svc.score(x_test_features,y_test)

0.7293260473588342

In [68]:
x2 = splitfun(x2)
x2

array([list(['AmericanAir', 'In', 'car', 'gng', 'to', 'DFW', 'Pulled', 'over', '1hr', 'ago', 'very', 'icy', 'roads', 'On', 'hold', 'with', 'AA', 'since', '1hr', "Can't", 'reach', 'arpt', 'for', 'AA2450', 'Wat', '2', 'do']),
       list(['AmericanAir', 'after', 'all', 'the', 'plane', 'didn', 't', 'land', 'in', 'identical', 'or', 'worse', 'conditions', 'at', 'GRK', 'according', 'to', 'METARs']),
       list(['SouthwestAir', "can't", 'believe', 'how', 'many', 'paying', 'customers', 'you', 'left', 'high', 'and', 'dry', 'with', 'no', 'reason', 'for', 'flight', 'Cancelled', 'Flightlations', 'Monday', 'out', 'of', 'BDL', 'Wow']),
       ...,
       list(['Nice', 'RT', 'VirginAmerica', 'The', 'man', 'of', 'steel', 'might', 'be', 'faster', 'but', 'we', 'have', 'WiFi', 'just', 'saying', 'ScienceBehindTheExperience', 'http', 't', 'co', 'FGRbpAZSiX']),
       list(['AmericanAir', 'Aww', 'Thanks', 'AA', 'DFW', 'was', 'on', 'GMA', 'up', 'here', 'this', 'AM', 'so', 'i', 'understand', 'Btw', 'A', 'A',

In [69]:
x2_final = [clean_review(words) for words in x2]
x2ffinal = [clean_review(words) for words in x2_final]
x2ffinal

[['americanair',
  'car',
  'gng',
  'dfw',
  'pulled',
  '1hr',
  'ago',
  'icy',
  'road',
  'hold',
  'aa',
  'since',
  '1hr',
  "can't",
  'reach',
  'arpt',
  'aa2450',
  'wat',
  '2'],
 ['americanair',
  'land',
  'identical',
  'worse',
  'condition',
  'grk',
  'according',
  'metars'],
 ['southwestair',
  "can't",
  'believe',
  'many',
  'pay',
  'customer',
  'left',
  'high',
  'dry',
  'reason',
  'cancel',
  'flightlations',
  'monday',
  'bdl',
  'wow'],
 ['usairways',
  'legitimately',
  'say',
  'would',
  'rather',
  'driven',
  'cross',
  'country',
  'flown',
  'us',
  'airway'],
 ['americanair', 'still', 'response', 'aa', 'great', 'job', 'guy'],
 ['united',
  'developer',
  'flying',
  'tmrw',
  'morn',
  'w',
  '45',
  'min',
  'layover',
  'earlier',
  '1',
  '5hr',
  'layover',
  'move'],
 ['usairways', 'hello', 'anyone'],
 ['usairways',
  'husainhaqqani',
  'mr',
  'husain',
  'u',
  'shld',
  'protest',
  'well',
  'one',
  'ur',
  'party',
  'member',
  'reh

In [70]:
test_docs = [" ".join(word) for word in x2ffinal]
test_docs[0]

"americanair car gng dfw pulled 1hr ago icy road hold aa since 1hr can't reach arpt aa2450 wat 2"

In [83]:
x2_test_features = count_vec.transform(test_docs)

In [84]:
y_pred = rfc.predict(x2_test_features)

In [85]:
y_pred

array(['negative', 'neutral', 'negative', ..., 'neutral', 'positive',
       'negative'], dtype='<U8')

In [86]:
np.savetxt(X = y_pred,fname= "twitter_pred.csv", fmt='%s')